In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#imports
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.optim.lr_scheduler import CosineAnnealingLR
from scipy.stats import qmc

In [ ]:
# Fourier Feature Embedding
class FourierFeatureLayer(nn.Module):
    def __init__(self, input_dim, num_features, scale=10.0):
        super(FourierFeatureLayer, self).__init__()
        self.B = nn.Parameter(scale * torch.randn(input_dim, num_features), requires_grad=False)

    def forward(self, x):
        x_proj = 2 * np.pi * x @ self.B
        return torch.cat([torch.sin(x_proj), torch.cos(x_proj)], dim=-1)

In [ ]:
# Define the Physics-Informed Neural Network (PINN)
class PINN(nn.Module):
    def __init__(self, num_features=50):
        super(PINN, self).__init__()
        self.fourier = FourierFeatureLayer(2, num_features)
        self.layers = nn.Sequential(
            nn.Linear(2 * num_features, 128),
            nn.Softplus(),
            nn.Linear(128, 128),
            nn.Softplus(),
            nn.Linear(128, 128),
            nn.Softplus(),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        x_ff = self.fourier(x)
        return self.layers(x_ff)

In [ ]:
# Define the PDE residual
class ConvectionDiffusionLoss(nn.Module):
    def __init__(self, epsilon, bx, by):
        super(ConvectionDiffusionLoss, self).__init__()
        self.epsilon = epsilon
        self.bx = bx
        self.by = by

    def forward(self, model, x):
        x.requires_grad = True
        y_pred = model(x)
        
        # Compute gradients
        dy_dx = torch.autograd.grad(y_pred, x, grad_outputs=torch.ones_like(y_pred), create_graph=True)[0]
        u = y_pred
        u_x = dy_dx[:, 0:1]
        u_y = dy_dx[:, 1:2]
        
        u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0][:, 0:1]
        u_yy = torch.autograd.grad(u_y, x, grad_outputs=torch.ones_like(u_y), create_graph=True)[0][:, 1:2]

        # Forcing function
        x1, y1 = x[:, 0:1], x[:, 1:2]
        f = (2 * self.epsilon * (-x1 + torch.exp(2 * (x1 - 1) / self.epsilon)) + x1 * y1**2 +
             6 * x1 * y1 - x1 * torch.exp(3 * (y1 - 1) / self.epsilon) - y1**2 * torch.exp(2 * (x1 - 1) / self.epsilon) +
             2 * y1**2 - 6 * y1 * torch.exp(2 * (x1 - 1) / self.epsilon) -
             2 * torch.exp(3 * (y1 - 1) / self.epsilon) + torch.exp(2 * x1 + 3 * y1 - 5 / self.epsilon))

        # Residual of the PDE
        residual = -self.epsilon * (u_xx + u_yy) + self.bx * u_x + self.by * u_y - f
        return torch.mean(residual**2)

In [ ]:
# Hyperparameters
epsilon = 1e-4
bx = 2
by = 3
epochs = 20000
learning_rate = 1e-3
num_features = 100

In [ ]:
# Model, loss, optimizer
model = PINN(num_features=num_features)
loss_fn = ConvectionDiffusionLoss(epsilon, bx, by)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=epochs)

In [ ]:
# Training points using Sobol sampling
n_train = 1000
sampler = qmc.Sobol(d=2)
x_train = torch.tensor(sampler.random_base2(m=14)[:n_train], dtype=torch.float32, requires_grad=True)

In [ ]:
# Boundary conditions
boundary_x = torch.cat([torch.zeros((1000, 1)), torch.rand((1000, 1))], dim=1)
boundary_y = torch.cat([torch.rand((1000, 1)), torch.zeros((1000, 1))], dim=1)
boundary = torch.cat([boundary_x, boundary_y], dim=0)

boundary_u = torch.zeros((boundary.shape[0], 1))

In [ ]:
# Training loop
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    # Compute PDE loss
    loss_pde = loss_fn(model, x_train)

    # Compute boundary loss
    u_pred_boundary = model(boundary)
    loss_boundary = torch.mean((u_pred_boundary - boundary_u) ** 2)

    # Total loss
    loss = loss_pde + 50 * loss_boundary
    loss.backward()
    optimizer.step()
    scheduler.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch}: PDE Loss = {loss_pde.item()}, Boundary Loss = {loss_boundary.item()}, Total Loss = {loss.item()}")

    # Progressive sampling: Increase training points every 1000 epochs
    if epoch % 1000 == 0 and n_train < 10000:
        n_train += 1000
        x_train = torch.tensor(sampler.random(n_train), dtype=torch.float32, requires_grad=True)


In [ ]:
# Load test data
test_data = pd.read_csv("/kaggle/input/casml-2024-scientific-machine-learning-challenge/test.csv")
test_points = torch.tensor(test_data[["x", "y"]].values, dtype=torch.float32)

In [ ]:
# Make predictions
model.eval()
predictions = model(test_points).detach().numpy()

In [ ]:
# Prepare submission
submission = test_data.copy()
submission["u"] = predictions
submission.to_csv("submission.csv", index=False)